In [1]:
import sys
sys.path.append('..')

In [2]:
from robustness_gym import *

In [3]:
import json
from fuzzywuzzy import fuzz
from nlp import list_datasets

## Cached Operations

In [4]:
# Load a few examples from the boolq dataset
dataset = Dataset.load_dataset('boolq', split='train[:1%]')

Checking /Users/krandiash/.cache/huggingface/datasets/494f4f058948bc745482f513922716c8bff83799bb20c7026e85a0026b643401.8462e3fa824dc775b18d0c33eb06f61781c1157028a832923a3b9547ee06f9c2.py for additional imports.
Lock 140652529281488 acquired on /Users/krandiash/.cache/huggingface/datasets/494f4f058948bc745482f513922716c8bff83799bb20c7026e85a0026b643401.8462e3fa824dc775b18d0c33eb06f61781c1157028a832923a3b9547ee06f9c2.py.lock
Found main folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/boolq/boolq.py at /usr/local/anaconda3/envs/mayanshell/lib/python3.8/site-packages/nlp/datasets/boolq
Found specific version folder for dataset https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/boolq/boolq.py at /usr/local/anaconda3/envs/mayanshell/lib/python3.8/site-packages/nlp/datasets/boolq/f9eaaad11e850927735893d5fba78b4064e30569b9c0337facd7e35f6c0697a8
Found script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/datasets/boolq/boolq.py to /usr/

In [5]:
dataset

Dataset(features: {'answer': Value(dtype='bool', id=None), 'index': Value(dtype='string', id=None), 'passage': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None)}, num_rows: 94, num_slices: 0)

In [6]:
# Return a dataset which caches the outputs of the operations applied
dataset = dataset.stow(
    cached_ops={
        # Apply Spacy separately to only the "question" key
        Spacy(): [['question']],
        # Apply the StripText operation to the "question" key and the "passage" key separately
        StripText(): [['question'], ['passage']]
    },
    load_from_cache_file=False,
)

Caching processed dataset at cache-26181064159052232777117048075389735138573308848957013099079102149242.arrow
100%|██████████| 3/3 [00:00<00:00, 22.13it/s]
Done writing 94 examples in 158257 bytes /Users/krandiash/Desktop/workspace/projects/robustness-gym/examples/tmp_z5y07xr.


In [7]:
print(dataset[0].keys())
print(dataset[0]['cache'].keys())
print(dataset[0]['cache']['Spacy'].keys())
print(dataset[0]['cache']['StripText'].keys())

dict_keys(['answer', 'index', 'passage', 'question', 'cache'])
dict_keys(['Spacy', 'StripText'])
dict_keys(['question'])
dict_keys(['passage', 'question'])


In [8]:
my_cached_op = CachedOperation(identifier='MyOp',  
                                apply_fn=lambda text_batch_a, text_batch_b: [fuzz.ratio(text_a, text_b) for text_a, text_b in zip(text_batch_a, text_batch_b)])

In [9]:
dataset = dataset.stow(cached_ops={
    my_cached_op: [['question', 'passage']],
})

Caching processed dataset at cache-1728072295086098938381636053840329254782020419366196562308104870007.arrow
100%|██████████| 3/3 [00:00<00:00, 111.95it/s]
Done writing 94 examples in 159009 bytes /Users/krandiash/Desktop/workspace/projects/robustness-gym/examples/tmp55fwjiuo.


In [10]:
# The outputs are cached, as expected
print(dataset[0]['cache']['MyOp'])

{'["question", "passage"]': 13}


In [11]:
# The keys are JSON-dumped, so easy to load back in
for keys, value in dataset[0]['cache']['MyOp'].items():
    print(json.loads(keys), value)

['question', 'passage'] 13
